In [302]:

import numpy as np

from scipy.ndimage import gaussian_filter
import subprocess
from soma import aims

from sulci.registration.spam import spam_register
from soma.aimsalgo import MorphoGreyLevel_S16

# Global static variables
_AIMS_BINARY_ONE = 32767
_dilation = 1
_threshold = 0
_nb_spam_subjects = 61

# Anatomist

import anatomist.api as ana
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt
# launching anatomist
a = ana.Anatomist()

# Define functions

In [303]:
def dilate(mask, radius=_dilation):
    """Makes a dilation radius _dilation, in mm
    """
    arr = mask.np
    # Binarization of mask
    arr[arr < 1] = 0
    arr[arr >= 1] = _AIMS_BINARY_ONE
    # Dilates initial volume of 10 mm
    morpho = MorphoGreyLevel_S16()
    dilate = morpho.doDilation(mask, radius)
    arr_dilate = dilate.np
    arr_dilate[arr_dilate >= 1] = 1
    return dilate

In [304]:
def realign_mi_register(spam_vol: aims.Volume_FLOAT, skel_vol: aims.Volume_S16):
    """Realigns skeleton mask to spam
    
    skel_data is the test aims volume"""
    
    spam_vol = aims.Volume_FLOAT(spam_vol)
    skel_vol2 = aims.Volume_S16(skel_vol)
    skel_vol = aims.Volume(skel_vol2.getSize(), 'FLOAT')
    skel_vol.copyHeaderFrom(skel_vol2.header())
    skel_vol.np[:] = skel_vol2.np
    
    # Reads skeleton file
    skel_vol.np[:] = (skel_vol.np > 0).astype(np.float32)
    
    # Makes binarization and dilation on spam
    mask_result = aims.Volume(spam_vol.getSize(), 'S16')
    mask_result.copyHeaderFrom(spam_vol.header())
    mask_result.np[:] = spam_vol.np
    mask_result[mask_result.np <= _threshold] = 0
    mask_result.np[:] = dilate(mask_result).np

    # Masks skeleton data with dilated spam
    skel_vol.np[mask_result.np <= 0] = 0
    skel_vol_masked = aims.Volume_FLOAT(skel_vol)
    aims.write(skel_vol, "/tmp/skel_before.nii.gz")
    skel_vol.np[:] = 100*gaussian_filter(skel_vol.np, sigma=2)
    print(np.unique(skel_vol.np))
    aims.write(skel_vol, "/tmp/skel_before_filtered.nii.gz")
    
    # Writes nifti files
    aims.write(spam_vol, "/tmp/spam.nii.gz")
    
    # Makes realignment
    subprocess.check_call("AimsMIRegister -r /tmp/spam.nii.gz  -t /tmp/skel_before_filtered.nii.gz --dir /tmp/transform.trm", shell=True)
    # print(out_tr.np)
    
    # Applies the realignment
    subprocess.check_call("AimsApplyTransform -i /tmp/skel_before.nii.gz -o /tmp/skel_realigned.nii.gz -m /tmp/transform.trm", shell=True)
    
    # loads realigned file:
    after_vol = aims.read("/tmp/skel_realigned.nii.gz")
    
    return after_vol, skel_vol_masked

In [305]:
def realign_spam_register(spam_vol: aims.Volume_FLOAT, skel_vol: aims.Volume_S16):
    """Realigns skeleton mask to spam
    
    skel_f is a file name of skeleton file"""
        
    spam_vol = aims.Volume_FLOAT(spam_vol)
    skel_vol = aims.Volume_S16(skel_vol)
    
    # Reads spam and skeleton files
    skel_vol.np[:] = (skel_vol.np > 0).astype(np.int16)

    # # Makes binarization and dilation on spam
    # mask_result = aims.Volume(spam_vol.getSize(), 'S16')
    # mask_result.copyHeaderFrom(spam_vol.header())
    # mask_result.np[:] = spam_vol.np
    # mask_result[mask_result.np <= _threshold] = 0
    # mask_result.np[:] = dilate(mask_result).np

    # # Masks skeleton data with dilated spam
    # skel_vol.np[mask_result.np <= 0] = 0
    aims.write(skel_vol, "/tmp/skel_before.nii.gz")
    
    # Reads initial spam volume
    spam_vol.np[:] = spam_vol.np/np.max(spam_vol.np[:])
    
    # Makes realignment
    out_tr = spam_register(spam_vol,
                        skel_vol,
                        do_mask=False,
                        eps=20.,
                        R_angle_var=np.pi / 8,
                        t_var=20.,
                        verbose=False,
                        in_log=False,
                        calibrate_distrib=30)
    aims.write(out_tr, '/tmp/transform.trm')
    
    # Applies the realignment
    subprocess.check_call(f"AimsApplyTransform -i /tmp/skel_before.nii.gz -o /tmp/skel_realigned.nii.gz -m /tmp/transform.trm", shell=True)
    
    # loads realigned file:
    after = aims.read("/tmp/skel_realigned.nii.gz")
    
    return after

# Toy model

In [306]:
# Initialiser les volumes aims et matrices numpy
spam_file = '/neurospin/dico/data/deep_folding/current/mask/2mm/regions/L/S.C.-sylv._left.nii.gz'
spam_vol = aims.read(spam_file, dtype="Volume_FLOAT")
spam_vol.np[:] = 0

skel_f = '/neurospin/dico/data/deep_folding/current/datasets/pclean/binarized_skeletons/L/Lbinarized_skeleton_ammon.nii.gz' 
skel_vol = aims.read(skel_f, dtype="Volume_S16")
skel_vol.np[:] = 0

spam_arr = np.zeros(spam_vol.np.shape, dtype=np.float32)
skel_arr = np.zeros(spam_vol.np.shape, dtype=np.int16)

In [307]:
spam_vol.header()

{ 'volume_dimension' : [ 96, 114, 96, 1 ], 'sizeX' : 96, 'sizeY' : 114, 'sizeZ' : 96, 'sizeT' : 1, 'referential' : '84b1989b-eb68-8665-0049-8feaf3c22679', 'disk_data_type' : 'S16', 'bits_allocated' : 16, 'data_type' : 'S16', 'scale_factor_applied' : 0, 'possible_data_types' : [ 'S16', 'FLOAT', 'DOUBLE' ], 'cal_min' : 0, 'cal_max' : 0, 'freq_dim' : 0, 'phase_dim' : 0, 'slice_dim' : 0, 'slice_code' : 0, 'slice_start' : 0, 'slice_end' : 0, 'slice_duration' : 0, 'storage_to_memory' : [ -1, 0, 0, 95, 0, -1, 0, 113, 0, 0, -1, 95, 0, 0, 0, 1 ], 'voxel_size' : [ 2, 2, 2, 1 ], 'tr' : 1, 'referentials' : [ 'Talairach-MNI template-SPM' ], 'transformations' : [ [ -1, 0, 0, 96, 0, -1, 0, 96, 0, 0, -1, 114, 0, 0, 0, 1 ] ], 'toffset' : 0, 'xyz_units' : 0, 'time_units' : 0, 'descrip' : '', 'aux_file' : '', 'nifti_type' : 1, 'object_type' : 'Volume', 'file_type' : 'NIFTI-1' }

In [308]:
skel_vol.header()

{ 'volume_dimension' : [ 96, 114, 96, 1 ], 'sizeX' : 96, 'sizeY' : 114, 'sizeZ' : 96, 'sizeT' : 1, 'referential' : '84b1989b-eb68-8665-0049-8feaf3c22679', 'disk_data_type' : 'S16', 'bits_allocated' : 16, 'data_type' : 'S16', 'scale_factor_applied' : 0, 'possible_data_types' : [ 'S16', 'FLOAT', 'DOUBLE' ], 'cal_min' : 0, 'cal_max' : 0, 'freq_dim' : 0, 'phase_dim' : 0, 'slice_dim' : 0, 'slice_code' : 0, 'slice_start' : 0, 'slice_end' : 0, 'slice_duration' : 0, 'storage_to_memory' : [ -1, 0, 0, 95, 0, -1, 0, 113, 0, 0, -1, 95, 0, 0, 0, 1 ], 'voxel_size' : [ 2, 2, 2, 1 ], 'tr' : 1, 'referentials' : [ 'Talairach-MNI template-SPM' ], 'transformations' : [ [ -1, 0, 0, 96, 0, -1, 0, 96, 0, 0, -1, 114, 0, 0, 0, 1 ] ], 'toffset' : 0, 'xyz_units' : 0, 'time_units' : 0, 'descrip' : '', 'aux_file' : '', 'nifti_type' : 1, 'object_type' : 'Volume', 'file_type' : 'NIFTI-1' }

In [309]:
spam_arr.shape

(96, 114, 96, 1)

In [310]:
# spam_arr[30, 30:70, 50] = 1
spam_arr[30:70, 50:55, 30, 0] = 100.
# spam_arr[30:70, 50:55, 60, 0] = 100.
spam_arr = gaussian_filter(spam_arr, sigma=1)
spam_vol.np[:] = spam_arr
# skel_arr[32, 35:75, 52] = 1
skel_arr[35:75, 52:57, 32, 0] = 1
# skel_arr[35:75, 52:57, 62, 0] = 1
skel_vol.np[:] = skel_arr


In [311]:
type(spam_vol)

soma.aims.Volume_FLOAT

In [312]:
spam_vol.header()


{ 'volume_dimension' : [ 96, 114, 96, 1 ], 'sizeX' : 96, 'sizeY' : 114, 'sizeZ' : 96, 'sizeT' : 1, 'referential' : '84b1989b-eb68-8665-0049-8feaf3c22679', 'disk_data_type' : 'S16', 'bits_allocated' : 16, 'data_type' : 'S16', 'scale_factor_applied' : 0, 'possible_data_types' : [ 'S16', 'FLOAT', 'DOUBLE' ], 'cal_min' : 0, 'cal_max' : 0, 'freq_dim' : 0, 'phase_dim' : 0, 'slice_dim' : 0, 'slice_code' : 0, 'slice_start' : 0, 'slice_end' : 0, 'slice_duration' : 0, 'storage_to_memory' : [ -1, 0, 0, 95, 0, -1, 0, 113, 0, 0, -1, 95, 0, 0, 0, 1 ], 'voxel_size' : [ 2, 2, 2, 1 ], 'tr' : 1, 'referentials' : [ 'Talairach-MNI template-SPM' ], 'transformations' : [ [ -1, 0, 0, 96, 0, -1, 0, 96, 0, 0, -1, 114, 0, 0, 0, 1 ] ], 'toffset' : 0, 'xyz_units' : 0, 'time_units' : 0, 'descrip' : '', 'aux_file' : '', 'nifti_type' : 1, 'object_type' : 'Volume', 'file_type' : 'NIFTI-1' }

In [313]:
np.max(spam_arr)

39.530056

In [314]:
print(np.unique(spam_vol, return_counts=True))
print(np.unique(skel_vol, return_counts=True))

(array([0.00000000e+00, 2.39699149e-10, 7.93774557e-09, 8.17744539e-09,
       9.67015410e-08, 1.04878993e-07, 2.70799802e-07, 2.78977240e-07,
       4.33386248e-07, 5.38265226e-07, 5.38265283e-07, 7.14533144e-07,
       1.25279837e-06, 1.68594488e-06, 1.68618465e-06, 1.77470872e-06,
       1.78288633e-06, 1.79082394e-06, 1.79106360e-06, 3.29901695e-06,
       3.47311516e-06, 3.57799399e-06, 3.57799422e-06, 9.23845801e-06,
       1.47851679e-05, 1.78248956e-05, 1.78248974e-05, 1.83631601e-05,
       1.83631637e-05, 2.43766208e-05, 4.14869864e-05, 4.23112033e-05,
       4.27397827e-05, 4.58891991e-05, 5.58308275e-05, 5.58387655e-05,
       5.75167724e-05, 5.75249505e-05, 5.87702816e-05, 5.90410855e-05,
       5.93039440e-05, 5.93118821e-05, 6.05449895e-05, 6.08239679e-05,
       6.10947682e-05, 6.11029463e-05, 1.12547452e-04, 1.18486889e-04,
       1.18486896e-04, 1.89625658e-04, 2.17151683e-04, 2.17151712e-04,
       2.35514861e-04, 3.12639866e-04, 5.04402677e-04, 5.05414966e-04,
     

In [315]:
after_vol = realign_spam_register(spam_vol, skel_vol)
# after_vol = realign_mi_register(spam_vol, skel_vol)

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


loading direct transformations
Output dimensions: 96, 114, 96
Output voxel size: 2, 2, 2 mm
Resampling carto_volume of S16...   0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 %


somanifti warning: the data of the saved Nifti file will be loaded in a different in-memory order when opened by somanifti.


In [316]:
np.unique(after_vol.np)

array([0, 1], dtype=int16)

In [317]:
skel_a = a.toAObject(skel_vol)
spam_a = a.toAObject(spam_vol)
after_a = a.toAObject(after_vol)
skel_a.setPalette("BLUE-lfusion")
after_a.setPalette("Reds")
w = a.createWindow('Coronal')
w.addObjects(spam_a)
w.addObjects(skel_a)
w.addObjects(after_a)

observable 0x57a332a99990(N9anatomist7AVolumeIsEE) could not be removed from observer 0x57a32e7138b8 (N9anatomist8Fusion2DE)
observable 0x57a330b106c0(N9anatomist7AVolumeIsEE) could not be removed from observer 0x57a32c041c98 (N9anatomist8Fusion2DE)


# Use of functions using S.C.-sylv._left

In [318]:
# spam_file = '/neurospin/dico/data/deep_folding/current/mask/2mm/regions/L/S.C.-sylv._left.nii.gz'
# skel_f = '/neurospin/dico/data/deep_folding/current/datasets/pclean/binarized_skeletons/L/Lbinarized_skeleton_ammon.nii.gz' 

# # Reads spam and skeleton files
# spam_vol = aims.read(spam_file, dtype="Volume_FLOAT")
# skel_vol = aims.read(skel_f)

In [319]:
# after_vol, skel_vol_masked = realign_mi_register(spam_vol, skel_vol)

In [320]:
# skel_a = a.toAObject(skel_vol_masked)
# spam_a = a.toAObject(spam_vol)
# after_a = a.toAObject(after_vol)
# skel_a.setPalette("BLUE-lfusion")
# after_a.setPalette("Reds")
# w = a.createWindow('Coronal')
# w.addObjects(spam_a)
# w.addObjects(skel_a)
# w.addObjects(after_a)